# CASE STUDY

## CODE

### 1)Import Packages

In [ ]:
import pypsa
import pandapower as pp
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import warnings
import subprocess
from shapely.errors import ShapelyDeprecationWarning
import logging

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pypsa.pf").setLevel(logging.WARNING)
plt.rc("figure", figsize=(10, 8))

In [ ]:
#os.chdir("C:\Users\82103\Documents\Market Integration and Sector Coupling\pypsa\CASE STUDYexercise_0126")
excel_file_path = "C:\\Users\\82103\\Documents\\Market Integration and Sector Coupling\\pypsa\\TEST\\data_Germany.xlsx"

### 2)Create a network and set Snapshots

In [ ]:
# Create a new PyPSA network
network= pypsa.Network()
network.set_snapshots(range(8760))  # Solve for a year 365*24

### 3)Add Buses

In [ ]:
# Read excel file which contains Non renewable generators data
bus_data = pd.read_excel(excel_file_path, sheet_name='buses')

In [ ]:
for index,row in bus_data.iterrows():
    network.add(
    "Bus",
    name=row['bus'],
    v_nom=row['v_nom'],
    carrier=row['carrier'],
    x=row['x'],
    y=row['y']
)
    
network.buses

### 4)Add generators

In [ ]:
generators = pd.read_excel(excel_file_path,sheet_name="generators" )

network.madd(
"Generator",
    names= generators.generator,
    bus=list(generators.bus),
    carrier=list(generators.carrier),
    p_nom=list(generators.p_nom),      
    p_nom_max=list(generators.p_nom_A),     
    #p_nom_max=list(generators.p_nom_B),    
    #p_nom_max=list(generators.p_nom_C),    
    p_nom_extendable=list(generators.p_nom_extandable),
    marginal_cost=list(generators.marginal_cost),
    efficiency=list(generators.efficiency),
    capital_cost=list(generators.capital_cost)
    ) 

network.generators

In [ ]:
PV_timeseries=pd.read_excel(excel_file_path,sheet_name='PV_timeseries')
wind_timeseries=pd.read_excel(excel_file_path,sheet_name='wind_timeseries')

In [ ]:
network.generators_t.p_max_pu["50_Solar"] = list(PV_timeseries.p_nom_pu_solar_50Hertz)
network.generators_t.p_max_pu["50_Offshore Wind"] = list(wind_timeseries.p_nom_pu_offshore_50Hertz)
network.generators_t.p_max_pu["50_Onshore Wind"] = list(wind_timeseries.p_nom_pu_onshore_50Hertz)

network.generators_t.p_max_pu["A_Solar"] = list(PV_timeseries.p_nom_pu_solar_Amprion)
network.generators_t.p_max_pu["A_Onshore Wind"] = list(wind_timeseries.p_nom_pu_Amprion)

network.generators_t.p_max_pu["TBW_Solar"] = list(PV_timeseries.p_nom_pu_TransnetBW)
network.generators_t.p_max_pu["TBW_Onshore Wind"] = list(wind_timeseries.p_nom_pu_TransnetBW)

network.generators_t.p_max_pu["Ten_Solar"] = list(PV_timeseries.p_nom_pu_solar_Tennet)
network.generators_t.p_max_pu["Ten_Offshore Wind"] = list(wind_timeseries.p_nom_pu_offshore_Tennet)
network.generators_t.p_max_pu["Ten_Onshore Wind"] = list(wind_timeseries.p_nom_pu_onshore_Tennet)

### 5)Add load

### Added by Parag

In [ ]:
'''
You can add your all loads in a timeseries in columns and add load using network.add function like mentioned in the following code:
To verify whether the timeseries of the load data (p_set) is added properly, use:  `network.loads_t.p_set`  this command
'''

"""
# Read excel file which contains Non renewable generators data
def add_consumers(filename, index_col):
    try:
        load = pd.read_(filename,index_col=index_col)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return

    for load_name in list(load.columns.unique()):
        network.add("Load",
        f"{load_name}",
        bus=load_name,
        p_set=load[load_name].tolist()
        )

    return network.loads_t.p_set.head(5)    
"""

In [ ]:
demand = pd.read_excel(excel_file_path,sheet_name="load_hourly" )
'''
network.add("Load", "Amprion", bus="Amprion", p_set=demand["Amprion"])
network.add("Load", "Tennet", bus="Tennet", p_set=demand["TenneT"])
network.add("Load", "TransnetBW", bus="TransnetBW", p_set=demand["TransnetBW"])
network.add("Load", "50Hertz", bus="50Hertz", p_set=demand["50Hertz"])
'''
#'''Scenario A
network.add("Load", "Amprion", bus="Amprion", p_set=demand["load_A_Amprion"])
network.add("Load", "Tennet", bus="Tennet", p_set=demand["load_A_TenneT"])
network.add("Load", "TransnetBW", bus="TransnetBW", p_set=demand["load_A_TransnetBW"])
network.add("Load", "50Hertz", bus="50Hertz", p_set=demand["load_A_50Hertz"])
#'''

'''Scenario B
network.add("Load", "Amprion", bus="Amprion", p_set=demand["load_B_Amprion"])
network.add("Load", "Tennet", bus="Tennet", p_set=demand["load_B_TenneT"])
network.add("Load", "TransnetBW", bus="TransnetBW", p_set=demand["load_B_TransnetBW"])
network.add("Load", "50Hertz", bus="50Hertz", p_set=demand["load_B_50Hertz"])
'''

'''Scenario C
network.add("Load", "Amprion", bus="Amprion", p_set=demand["load_C_Amprion"])
network.add("Load", "Tennet", bus="Tennet", p_set=demand["load_C_TenneT"])
network.add("Load", "TransnetBW", bus="TransnetBW", p_set=demand["load_C_TransnetBW"])
network.add("Load", "50Hertz", bus="50Hertz", p_set=demand["load_C_50Hertz"])
'''

In [ ]:
network.loads_t.p_set.plot(figsize=(9,3), ylabel="MW")

### 6)Add Lines

In [ ]:
lines=pd.read_excel(excel_file_path,sheet_name='lines',header=0)
lines.head()

In [ ]:
for X, row in lines.iterrows():
    network.add("Line",
    name=row['name'],
    bus0=row['bus0'],
    bus1=row['bus1'],
    s_nom=row['s_nom'],
    x=row['efficiency'],
    s_nom_extendable=row['s_nom_extendable'] 
    )
network.lines

### 7)Add links

links = pd.read_excel(excel_file_path, sheet_name='links')
links.head()

for X, row in links.iterrows():
    network.add("Link",
    name=row['name'],
    bus0=row['bus0'],
    bus1=row['bus1'],
    p_nom=row['p_nom'],
    p_nom_extendable=row['p_nom_extendable'],
    efficiency=row['efficiency'],
    capital_cost=row['capital_cost'] )
network.links

### 8)Add carriers

In [ ]:
Carriers = pd.read_excel(excel_file_path,sheet_name="co2_emissions" )
Carriers.tail()

In [ ]:
for X, row in Carriers.iterrows():
    network.add(
        "Carrier",
        name=row['name'],
        co2_emissions=row['co2_emissions']
    )
network.carriers

In [ ]:
network.add("Store", "battery storage1", bus="50Hertz", e_cyclic=True, e_nom=10000000.0)
network.add("Store", "battery storage2", bus="Amprion", e_cyclic=True, e_nom=10000000.0)
network.add("Store", "battery storage3", bus="Tennet", e_cyclic=True, e_nom=10000000.0)
network.add("Store", "battery storage4", bus="TransnetBW", e_cyclic=True, e_nom=10000000.0)

In [ ]:
network.add(
    "StorageUnit",
    "battery storage1",
    bus="50Hertz",
    p_nom=100000000,
    max_hours=10,  # energy storage in terms of hours at full power
)

In [ ]:
network.add(
    "StorageUnit",
    "battery storage2",
    bus="Amprion",
    p_nom=100000000,
    max_hours=10,  # energy storage in terms of hours at full power
)

In [ ]:
network.add(
    "StorageUnit",
    "battery storage3",
    bus="Tennet",
    p_nom=100000000,
    max_hours=10,  # energy storage in terms of hours at full power
)

In [ ]:
network.add(
    "StorageUnit",
    "battery storage4",
    bus="TransnetBW",
    p_nom=100000000,
    max_hours=10,  # energy storage in terms of hours at full power
)

### 9)CO2 emissions

network.add("GlobalConstraint", "co2_limit", sense="<=", constant=438000000)

## RESULTS

### 1)Optimize

In [ ]:
network.optimize(solver_name='glpk')

### 2)Margianl price

In [ ]:
network.buses_t.marginal_price

In [ ]:
network.generators.p_nom_opt.plot.bar(ylabel="MW", figsize=(8, 3))

In [ ]:
network.generators_t.p.sum().plot.bar(ylabel="Total generation MW")

### 3)Lines snapshot

In [ ]:
network.lines_t.p0

### 4)Gen assign

In [ ]:
gen = network.generators.assign(g=network.generators_t.p.mean()).groupby(["bus", "carrier"]).g.sum()
gen

### 5)Flow

In [ ]:
flow = pd.Series(10, index=network.branches().index)
flow

### 6)Geo. graph

In [ ]:
plot_graph=network.plot(
    bus_sizes=gen/20000,
    bus_colors={"biomass":"black","coal":"blue","gas":"pink","hydro":"cadetblue","lignite":"yellow","solar":"orange","wind":"midnightblue"},
    margin=1,
    line_widths=3,
    link_widths=0,
    flow=flow,
    color_geomap=True,
    projection=ccrs.EqualEarth(),
    line_colors=network.lines_t.p0.mean().abs(),
)
plt.colorbar(plot_graph[2], fraction=0.04, pad=0.004, label="Flow in MW")
plt.show()

In [ ]:
# Your DataFrame df
df = pd.concat(
    [
        network.generators_t.p.loc[0],
        network.links_t.p0.loc[0],
        network.loads_t.p.loc[0],
    ],
    keys=["Generators", "Links", "Line"],
    names=["Component", "index"],
).reset_index(name="Production")

# Plotting using PyPSA
fig, ax = plt.subplots(figsize=(50, 10))

# Plot generators
df_generators = df[df["Component"] == "Generators"]
ax.bar(df_generators["index"], df_generators["Production"], label="Generators")

# Plot links
df_links = df[df["Component"] == "Links"]
ax.bar(df_links["index"], df_links["Production"], label="Links")

# Plot loads
df_loads = df[df["Component"] == "Line"]
ax.bar(df_loads["index"], df_loads["Production"], label="Line")

# Customize the plot
ax.set_xlabel("Component")
ax.set_ylabel("Production (MW)")
ax.set_title("Power Production by Component")
ax.legend()

# Show the plot
plt.show()


## ADDITIONAL

### -)Storages

In [ ]:
"""
def add_stores(filename, header):
    try:
        stores = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    for index, row in stores.iterrows():
        network.add(
            "Store",
            name=row['name'],
            e_initial=row['e_initial'],
            e_nom=row['e_nom'],
            marginal_cost=row['marginal_cost'],
            bus=row['bus'],
            e_cyclic=row['e_cyclic'],
            e_nom_extendable=row['e_nom_extendable'],
        )
    return network.stores
"""

add_stores('stores.csv',0)

### -)Global constraints

In [ ]:
#network.lopf()